### Import Packages

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets

### Load Dataset

In [2]:
train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

### Dataloader

In [3]:
batch_size = 100
n_iters = 3000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [4]:
input_size=28
sequence_length =28
num_layers=2
hidden_size=256

learning_rate = 0.001
num_epochs = 5

num_classes =10
batch_size = 64

### GRU

In [7]:
class SimpleGRU(nn.Module):
    def __init__(self, input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, num_classes=num_classes, sequence_length=sequence_length):
        super(SimpleGRU, self).__init__()
        self.hidden_size  = hidden_size
        self.num_layers = num_layers
        
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size * sequence_length, num_classes)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        
        out,_ = self.gru(x, h0)
        out = out.reshape(out.shape[0], -1)
        out = self.fc1(out)
        return out

In [6]:
model = SimpleGRU()
criterion = nn.CrossEntropyLoss()
learning_rate = 0.1

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

In [9]:
# Check if a GPU is available and use it, otherwise use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images[:, 0, :, :].to(device)  # Flatten the sequence dimension
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print the loss after each batch
        if (i+1) % 100 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}")

# Evaluate the model on the test dataset
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images[:, 0, :, :].to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")

Epoch [1/5], Step [100/600], Loss: 0.7371
Epoch [1/5], Step [200/600], Loss: 0.4305
Epoch [1/5], Step [300/600], Loss: 0.3741
Epoch [1/5], Step [400/600], Loss: 0.3355
Epoch [1/5], Step [500/600], Loss: 0.3364
Epoch [1/5], Step [600/600], Loss: 0.3723
Epoch [2/5], Step [100/600], Loss: 0.2314
Epoch [2/5], Step [200/600], Loss: 0.3558
Epoch [2/5], Step [300/600], Loss: 0.3904
Epoch [2/5], Step [400/600], Loss: 0.3293
Epoch [2/5], Step [500/600], Loss: 0.3355
Epoch [2/5], Step [600/600], Loss: 0.2713
Epoch [3/5], Step [100/600], Loss: 0.2548
Epoch [3/5], Step [200/600], Loss: 0.1124
Epoch [3/5], Step [300/600], Loss: 0.2738
Epoch [3/5], Step [400/600], Loss: 0.2465
Epoch [3/5], Step [500/600], Loss: 0.1412
Epoch [3/5], Step [600/600], Loss: 0.2304
Epoch [4/5], Step [100/600], Loss: 0.0801
Epoch [4/5], Step [200/600], Loss: 0.1738
Epoch [4/5], Step [300/600], Loss: 0.1286
Epoch [4/5], Step [400/600], Loss: 0.1623
Epoch [4/5], Step [500/600], Loss: 0.1307
Epoch [4/5], Step [600/600], Loss: